## 01_backtesting-py_simple.ipynb
- **Goal:** Evaluate a long-only SMA crossover strategy on BTCUSDT hourly data with the `backtesting.py` framework.
- **Data preparation:** Retrieves candles through `src.data.data_pipeline.get_historical_data`, then converts the Polars DataFrame to pandas for compatibility with `Backtest`.
- **Strategy logic:** Implements `SmaCross_bt`, which tracks short/long simple moving averages, buys on upward crossovers, and exits on downward crossovers.
- **Backtest run:** Executes `Backtest` with $1M starting cash, 0.2% commission, and exclusive orders; prints the resulting performance statistics and generates the HTML equity plot.
- **Optimization experiments:** Runs a 300-trial Optuna study optimizing both return and Sharpe ratio (multi-objective) followed by a single-objective variant that combines the same metrics, then re-runs the backtest with the chosen parameters (`n_short=47`, `n_long=67`).

In [1]:
import os
import sys
# Add src folder to Python path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

## Load data

In [2]:
from src.data.data_pipeline import get_historical_data
from src.utils.utils import polars_to_pandas

In [3]:
df = get_historical_data(download=False)
df.head()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_base_vol,taker_quote_vol,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,str
2024-09-24 07:00:00,63305.43,63569.99,63300.0,63559.92,717.69816,2024-09-24 07:59:59.999,4.5550e7,98864,390.39104,2.4774e7,"""0"""
2024-09-24 08:00:00,63559.93,63948.0,63540.0,63851.05,995.90843,2024-09-24 08:59:59.999,6.3463e7,131479,496.91258,3.1667e7,"""0"""
2024-09-24 09:00:00,63851.05,63893.51,63484.0,63524.5,729.48652,2024-09-24 09:59:59.999,4.6436e7,123317,292.08143,1.8592e7,"""0"""
2024-09-24 10:00:00,63524.5,63883.0,63504.1,63526.0,618.45246,2024-09-24 10:59:59.999,3.9361e7,101884,370.79941,2.3604e7,"""0"""
2024-09-24 11:00:00,63526.01,63600.0,63380.0,63478.34,591.1906,2024-09-24 11:59:59.999,3.7530e7,92450,260.37361,1.6529e7,"""0"""


## Defining strategy

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from backtesting import Backtest

class SmaCross_bt(Strategy):
    """
    A simple SMA Cross strategy:
    - If short SMA crosses above long SMA, enter a long position.
    - If short SMA crosses below long SMA, close the position.
    """

    n_short = 30
    n_long = 100

    def init(self):
        # Convert the series to indicators used by backtesting.py
        price = self.data.Close
        self.sma_short = self.I(SMA, price, self.n_short)
        self.sma_long = self.I(SMA, price, self.n_long)

    def next(self):
        # If short SMA crosses above long SMA, and not already in a trade:
        if crossover(self.sma_short, self.sma_long):
            self.buy()

        # If short SMA crosses below long SMA, close any open position:
        elif crossover(self.sma_long, self.sma_short):
            self.position.close()

/Users/ezequielmrivero/GITHUB/tradelab_repositories/Crypto-Backtester-Duel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ezequielmrivero/GITHUB/tradelab_repositories/Crypto-Backtester-Duel/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
def run_bt_sma_backtest(df_pl) -> None:
    df_pd = polars_to_pandas(df_pl)
    bt = Backtest(df_pd, SmaCross_bt, cash=1_000_000, commission=0.002, exclusive_orders=True)
    stats = bt.run()
    return stats, bt

stats, bt = run_bt_sma_backtest(df)
print(stats)
bt.plot()

Start                     2024-09-24 07:00:00
End                       2025-09-24 06:00:00
Duration                    364 days 23:00:00
Exposure Time [%]                    55.73059
Equity Final [$]                1094161.98954
Equity Peak [$]                 1535359.05996
Commissions [$]                  244949.18046
Return [%]                             9.4162
Buy & Hold Return [%]                71.21324
Return (Ann.) [%]                      9.3893
Volatility (Ann.) [%]                36.43405
CAGR [%]                              9.41732
Sharpe Ratio                          0.25771
Sortino Ratio                         0.46411
Calmar Ratio                          0.26886
Alpha [%]                           -27.27643
Beta                                  0.51525
Max. Drawdown [%]                   -34.92262
Avg. Drawdown [%]                    -2.13122
Max. Drawdown Duration      280 days 16:00:00
Avg. Drawdown Duration        7 days 05:00:00
# Trades                          

GridPlot(id='p1356', ...)

In [6]:
import optuna
from backtesting import Backtest

# Objective function for Optuna
def multi_objective(trial):
    # 1) Suggest parameters
    n_short = trial.suggest_int('n_short', 5, 50)
    # ensure n_long > n_short
    n_long  = trial.suggest_int('n_long', n_short + 1, 200)

    # 2) Update strategy class vars
    SmaCross_bt.n_short = n_short
    SmaCross_bt.n_long  = n_long

    # 3) Run backtest
    stats, _ = run_bt_sma_backtest(df)

    # 4) Extract objectives
    ret    = stats['Return [%]']    # e.g. 12.34
    sharpe = stats['Sharpe Ratio']  # e.g. 1.23

    # Optuna multi-objective: return a tuple
    return ret, sharpe

# Create a multi-objective study: maximize both return and sharpe
study = optuna.create_study(
    directions=['maximize', 'maximize'],
    study_name='sma_cross_multiobj'
)

# Run optimization
study.optimize(multi_objective, n_trials=300)
print("Number of Pareto-optimal trials:", len(study.best_trials))
for t in study.best_trials:
    print(f"  trial#{t.number}: Return={t.values[0]:.2f}%, Sharpe={t.values[1]:.2f}, "
          f"n_short={t.params['n_short']}, n_long={t.params['n_long']}")

[I 2025-10-10 16:58:26,075] A new study created in memory with name: sma_cross_multiobj
[I 2025-10-10 16:58:26,129] Trial 0 finished with values: [-3.4240749500000383, -0.11041508101388665] and parameters: {'n_short': 16, 'n_long': 149}.
[I 2025-10-10 16:58:26,180] Trial 1 finished with values: [-3.937433167999913, -0.1290082212252808] and parameters: {'n_short': 30, 'n_long': 194}.
[I 2025-10-10 16:58:26,228] Trial 2 finished with values: [22.732213093999984, 0.5591092197357559] and parameters: {'n_short': 24, 'n_long': 82}.
[I 2025-10-10 16:58:26,272] Trial 3 finished with values: [-5.7899156960000395, -0.1850122061957469] and parameters: {'n_short': 15, 'n_long': 196}.
[I 2025-10-10 16:58:26,319] Trial 4 finished with values: [-22.029611524000135, -0.8798424724226017] and parameters: {'n_short': 34, 'n_long': 42}.
[I 2025-10-10 16:58:26,364] Trial 5 finished with values: [5.12832161600003, 0.1421354586548128] and parameters: {'n_short': 48, 'n_long': 116}.
[I 2025-10-10 16:58:26,409

Number of Pareto-optimal trials: 1
  trial#141: Return=33.78%, Sharpe=0.74, n_short=42, n_long=70


In [7]:
import optuna
from backtesting import Backtest

# Objective function for Optuna
def single_objective(trial):
    # 1) Suggest parameters
    n_short = trial.suggest_int('n_short', 5, 50)
    # ensure n_long > n_short
    n_long  = trial.suggest_int('n_long', n_short + 1, 200)

    # 2) Update strategy class vars
    SmaCross_bt.n_short = n_short
    SmaCross_bt.n_long  = n_long

    # 3) Run backtest
    stats, _ = run_bt_sma_backtest(df)

    # 4) Extract objectives
    ret    = stats['Return [%]']    # e.g. 12.34
    sharpe = stats['Sharpe Ratio']  # e.g. 1.23

    return ret + 10*sharpe

# Create a multi-objective study: maximize both return and sharpe
study = optuna.create_study(
    direction='maximize',
    study_name='sma_cross_singleobj'
)

# Run optimization
study.optimize(single_objective, n_trials=300)
print("Number of Pareto-optimal trials:", len(study.best_trials))
for t in study.best_trials:
    print(f"  trial#{t.number}: "
          f"n_short={t.params['n_short']}, n_long={t.params['n_long']}")

[I 2025-10-10 16:58:40,620] A new study created in memory with name: sma_cross_singleobj
[I 2025-10-10 16:58:40,667] Trial 0 finished with value: 11.802903799245906 and parameters: {'n_short': 45, 'n_long': 193}. Best is trial 0 with value: 11.802903799245906.
[I 2025-10-10 16:58:40,719] Trial 1 finished with value: -142.54315406014126 and parameters: {'n_short': 37, 'n_long': 38}. Best is trial 0 with value: 11.802903799245906.
[I 2025-10-10 16:58:40,765] Trial 2 finished with value: -2.43477086952316 and parameters: {'n_short': 15, 'n_long': 76}. Best is trial 0 with value: 11.802903799245906.
[I 2025-10-10 16:58:40,811] Trial 3 finished with value: -14.410560159805957 and parameters: {'n_short': 37, 'n_long': 54}. Best is trial 0 with value: 11.802903799245906.
[I 2025-10-10 16:58:40,856] Trial 4 finished with value: -6.749091534147247 and parameters: {'n_short': 48, 'n_long': 131}. Best is trial 0 with value: 11.802903799245906.
[I 2025-10-10 16:58:40,902] Trial 5 finished with val

Number of Pareto-optimal trials: 4
  trial#88: n_short=29, n_long=66
  trial#227: n_short=29, n_long=66
  trial#247: n_short=29, n_long=66
  trial#250: n_short=29, n_long=66


In [8]:
SmaCross_bt.n_short = 47
SmaCross_bt.n_long  = 67
stats, bt = run_bt_sma_backtest(df)
print(stats)
bt.plot()

Start                     2024-09-24 07:00:00
End                       2025-09-24 06:00:00
Duration                    364 days 23:00:00
Exposure Time [%]                    54.58904
Equity Final [$]                1266902.54922
Equity Peak [$]                 1639846.53594
Commissions [$]                  419503.44078
Return [%]                           26.69025
Buy & Hold Return [%]                72.90673
Return (Ann.) [%]                    26.60839
Volatility (Ann.) [%]                43.08171
CAGR [%]                             26.69368
Sharpe Ratio                          0.61763
Sortino Ratio                          1.3588
Calmar Ratio                          0.94655
Alpha [%]                            -11.6697
Beta                                  0.52615
Max. Drawdown [%]                   -28.11102
Avg. Drawdown [%]                     -1.7937
Max. Drawdown Duration      280 days 16:00:00
Avg. Drawdown Duration        6 days 05:00:00
# Trades                          

GridPlot(id='p1736', ...)